In [2]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
#%matplotlib inline 

import cv2
import os

import tensorflow as tf

import keras

In [15]:
os.chdir("/rds/general/user/ft824/home/ML_BreakHis/scr")

In [4]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

#download pretrained weights 
#model = ResNet50(weights='imagenet', include_top=False)


In [23]:
# get pretrained weight without last layer
resnet_weights_path = '/rds/general/user/ft824/home/.keras/models/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

CHANNELS = 3
IMAGE_RESIZE = 224
NUM_CLASSES = 2 # change this to match your number of output classes
DENSE_LAYER_ACTIVATION = 'sigmoid'  # use 'softmax' for categorical classification
RESNET50_POOLING_AVERAGE = 'avg'  
OBJECTIVE_FUNCTION = 'categorical_crossentropy'

# Common accuracy metric for all outputs, but can use different metrics for different output
LOSS_METRICS = ['accuracy']

# EARLY_STOP_PATIENCE must be < NUM_EPOCHS
NUM_EPOCHS = 10
EARLY_STOP_PATIENCE = 3

# These steps value should be proper FACTOR of no.-of-images in train & valid folders respectively
STEPS_PER_EPOCH_TRAINING = 10
STEPS_PER_EPOCH_VALIDATION = 10

#BATCH_SIZE sould be FACTOR of no of img in train and validation
BATCH_SIZE_TRAINING = 32
BATCH_SIZE_VALIDATION = 16

In [6]:
# Build the model
model = Sequential()

# Add pre-trained ResNet50 as the base (without the top classifier layer)
model.add(ResNet50(
    include_top=False,
    pooling=RESNET50_POOLING_AVERAGE,
    weights=resnet_weights_path,
    input_shape=(224, 224, 3)  # or your image size
))

# Freeze the base model, not to train first layer
model.layers[0].trainable = False

# Add output layer for classification
model.add(Dense(NUM_CLASSES, activation=DENSE_LAYER_ACTIVATION))

In [7]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 2048)           │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │         4,098 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,591,810 (90.00 MB)

 Trainable params: 4,098 (16.01 KB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [8]:
from tensorflow.keras.optimizers import SGD

# Define optimizer
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)

# Compile model
model.compile(optimizer=sgd, loss=OBJECTIVE_FUNCTION, metrics=LOSS_METRICS)


/rds/general/user/ft824/home/anaconda3/envs/breakhis/lib/python3.10/site-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [20]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input

# Load your CSV files
train_df = pd.read_csv('../data/new_train.csv')
augment_train = pd.read_csv('../data/augmented_dataset.csv')
train_df = pd.concat([train_df, augment_train], axis=0, ignore_index=True)

test_df = pd.read_csv('../data/new_test.csv')

image_size = IMAGE_RESIZE  # for ResNet50


# Define the ImageDataGenerator with preprocessing
datagen = ImageDataGenerator(preprocessing_function=preprocess_input)


# Training generator
train_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='filepath',    # column with image file paths
    y_col='label',       # column with image labels
    target_size=(image_size, image_size),  # resizing to match ResNet50 input size
    batch_size=BATCH_SIZE_TRAINING,
    class_mode='categorical' # multi-class classification
)

# Test generator
test_generator = datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='filepath',    # column with image file paths
    y_col='label',       # column with image labels
    target_size=(image_size, image_size),  # resizing to match ResNet50 input size
    batch_size=BATCH_SIZE_VALIDATION,
    class_mode='categorical'
)

Found 4458 validated image filenames belonging to 2 classes.
Found 1483 validated image filenames belonging to 2 classes.


In [21]:
(BATCH_SIZE_TRAINING, len(train_generator), BATCH_SIZE_VALIDATION, len(test_generator))

(32, 140, 16, 93)

In [ ]:
##check for missign files
print(train_df['filepath'].head())
missing = train_df[~train_df['filepath'].apply(os.path.exists)]
print(f"Missing files: {len(missing)}")
print(missing.head())


0    ../../.cache/kagglehub/datasets/ambarish/break...
1    ../../.cache/kagglehub/datasets/ambarish/break...
2    ../../.cache/kagglehub/datasets/ambarish/break...
3    ../../.cache/kagglehub/datasets/ambarish/break...
4    ../../.cache/kagglehub/datasets/ambarish/break...
Name: filepath, dtype: object
Missing files: 0
Empty DataFrame
Columns: [filepath, label, magnification, tumor_subtype, Unnamed: 0]
Index: []


In [22]:
# Early stopping & checkpointing the best model in ../working dir & restoring that as our model for prediction
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

cb_early_stopper = EarlyStopping(monitor = 'val_loss', patience = EARLY_STOP_PATIENCE)
cb_checkpointer = ModelCheckpoint(filepath = '../working/best.hdf5', monitor = 'val_loss', save_best_only = True, mode = 'auto')

In [25]:
fit_history = model.fit(
        train_generator,
        steps_per_epoch=STEPS_PER_EPOCH_TRAINING,
        epochs = NUM_EPOCHS,
        validation_data=test_generator,
        validation_steps=STEPS_PER_EPOCH_VALIDATION,
        callbacks=[cb_checkpointer, cb_early_stopper]
)
model.load_weights("/working/best.hdf5")

/rds/general/user/ft824/home/anaconda3/envs/breakhis/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 17s/step - accuracy: 0.5872 - loss: 2.4717 

ValueError: The following argument(s) are not supported: ['options']